In [20]:
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import ndarrays_to_parameters, Context
from flwr.server import ServerApp, ServerConfig
from flwr.server import ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr.common import Metrics, NDArrays, Scalar
from typing import List, Tuple, Dict, Optional

In [21]:
import tensorflow as tf
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
x_train = x_train.reshape(-1, 28 * 28)  # Flatten images
x_test = x_test.reshape(-1, 28 * 28)



In [22]:
import numpy as np

def exclude_digits(x_data, y_data, excluded_digits):
    mask = ~np.isin(y_data, excluded_digits)  # Create a mask for non-excluded digits
    x_filtered = x_data[mask]  # Filter input data
    y_filtered = y_data[mask]  # Filter labels
    return x_filtered, y_filtered

### Create Train dataset

In [23]:
x_train_1, y_train_1  = exclude_digits(x_train, y_train, excluded_digits=[1, 3, 7])
print("y_train_1 : ",set(y_train_1))


x_train_2, y_train_2 = exclude_digits(x_train, y_train, excluded_digits=[2, 5, 8])
print("y_train_2 : ",set(y_train_2))

x_train_3, y_train_3 = exclude_digits(x_train, y_train, excluded_digits=[4, 6, 9])
print("y_train_3 : ",set(y_train_3))


y_train_1 :  {0, 2, 4, 5, 6, 8, 9}
y_train_2 :  {0, 1, 3, 4, 6, 7, 9}
y_train_3 :  {0, 1, 2, 3, 5, 7, 8}


In [24]:
train_sets = [(x_train_1, y_train_1 ),(x_train_2, y_train_2),(x_train_3, y_train_3)]

### Create Test dataset

In [25]:
x_test_1, y_test_1  = exclude_digits(x_test, y_test, excluded_digits=[1, 3, 7])
print("y_test_1 : ",set(y_test_1))

x_test_2, y_test_2  = exclude_digits(x_test, y_test, excluded_digits=[2, 5, 8])
print("y_test_2 : ",set(y_test_2))

x_test_3, y_test_3  = exclude_digits(x_test, y_test, excluded_digits=[4,6,9])
print("y_test_3 : ",set(y_test_3))


y_test_1 :  {0, 2, 4, 5, 6, 8, 9}
y_test_2 :  {0, 1, 3, 4, 6, 7, 9}
y_test_3 :  {0, 1, 2, 3, 5, 7, 8}


In [26]:
test_sets = [(x_test_1, y_test_1),(x_test_2, y_test_2),(x_test_3, y_test_3)]

### Prepare model training
- Sample model
- training method
- Evaluation method

In [27]:
class SimpleNN(tf.keras.Model):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

In [28]:
def train_model(model,pth_x,pth_y):
    batch_size = 64
    epochs = 20
    num_batches = len(pth_x) // batch_size
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    # Convert labels to one-hot encoding
    pth_y_onehot = tf.keras.utils.to_categorical(pth_y, num_classes=10)

    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for i in range(num_batches):
            # Get a batch of data
            start = i * batch_size
            end = start + batch_size
            x_batch = pth_x[start:end]
            y_batch = pth_y_onehot[start:end]
            
            with tf.GradientTape() as tape:
                predictions = model(x_batch, training=True)  # Forward pass
                loss = loss_fn(y_batch, predictions)        # Compute loss
            

            gradients = tape.gradient(loss, model.trainable_variables) 
        
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Update weights

            if i % 200 == 0:  # Print progress every 200 batches
                print(f"Batch {i}/{num_batches}, Loss: {loss.numpy():.4f}")

In [29]:
def evaluate_model(model,x_test,y_test):
    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=10)

    # Evaluate the model
    test_loss = loss_fn(y_test_onehot, model(x_test))
    print("test_loss : ",test_loss)
    test_accuracy = tf.keras.metrics.categorical_accuracy(y_test_onehot, model(x_test))
    print("test_accuracy : ",test_accuracy)
    return test_loss, test_accuracy

### Fed learning

In [30]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainset, testset):
        self.net = net
        self.trainset = trainset
        self.testset = testset

    # Train the model
    def fit(self, parameters, config):
        self.net.set_weights(parameters)
        pth_x,pth_y = self.trainset # (x,y)
        train_model(self.net,pth_x,pth_y )
        return self.net.get_weights(), len(pth_y), {}

    # Test the model
    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        self.net.set_weights(parameters)
        loss, accuracy = evaluate_model(self.net, self.testset)
        return loss, len(self.testset), {"accuracy": accuracy}

In [31]:
# Client function
def client_fn(context: Context) -> Client:
    net = SimpleNN()
    partition_id = int(context.node_config["partition-id"])
    client_train = train_sets[int(partition_id)]
    client_test = test_sets[int(partition_id)]
    return FlowerClient(net, client_train, client_test).to_client()

In [32]:
client = ClientApp(client_fn)

In [14]:
# test code 
net = SimpleNN()
_, accuracy137 = evaluate_model(net, x_test_1,y_test_1)
print("test accuracy on [1,3,7]: ", accuracy137)

2025-06-06 23:07:20.853120: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-06-06 23:07:20.853398: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-06-06 23:07:20.853406: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1749226040.853430 9868729 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1749226040.853484 9868729 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


test_loss :  tf.Tensor(2.4357412, shape=(), dtype=float32)
test_accuracy :  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(6827,), dtype=float32)
test accuracy on [1,3,7]:  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(6827,), dtype=float32)


### Define evaluate for model testing
- The evaluate method evaluates the performance of the neural network model using the provided parameters and the test dataset (testset).

In [15]:
def evaluate(server_round, parameters, config):
    net = SimpleNN()
    net.set_weights(parameters)

    _, accuracy137 = evaluate_model(net, x_test_1,y_test_1)
    _, accuracy258 = evaluate_model(net, x_test_2,y_test_2)
    _, accuracy469 = evaluate_model(net, x_test_3,y_test_3)

    print("test accuracy on [1,3,7]: ", accuracy137)
    print("test accuracy on [2,5,8]: ", accuracy258)
    print("test accuracy on [4,6,9]: ", accuracy469)


The federated averaging strategy (`strategy.FedAvg`) is created for federated learning.

In [16]:
net = SimpleNN()
params = ndarrays_to_parameters(net.get_weights())

def server_fn(context: Context):
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=params,
        evaluate_fn=evaluate,
    )
    config=ServerConfig(num_rounds=3)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )

In [17]:
server = ServerApp(server_fn=server_fn)

In [18]:
from logging import ERROR
backend_setup = {"init_args": {"logging_level": ERROR, "log_to_driver": False}}


In [19]:
# Initiate the simulation passing the server and client apps
# Specify the number of super nodes that will be selected on every round
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=3,
    #backend_config=backend_setup,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]


INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


test_loss :  tf.Tensor(2.429636, shape=(), dtype=float32)
test_accuracy :  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(6827,), dtype=float32)
test_loss :  tf.Tensor(2.4564862, shape=(), dtype=float32)
test_accuracy :  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(7102,), dtype=float32)
test_loss :  tf.Tensor(2.3757248, shape=(), dtype=float32)


INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


test_accuracy :  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(7051,), dtype=float32)
test accuracy on [1,3,7]:  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(6827,), dtype=float32)
test accuracy on [2,5,8]:  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(7102,), dtype=float32)
test accuracy on [4,6,9]:  tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(7051,), dtype=float32)


...

(ClientAppActor pid=62573) Epoch 1/20
(ClientAppActor pid=62573) Batch 0/660, Loss: 2.1752
(ClientAppActor pid=62574) Epoch 1/20 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(ClientAppActor pid=62572) Batch 200/668, Loss: 0.2334 [repeated 4x across cluster]
(ClientAppActor pid=62573) Batch 600/660, Loss: 0.1795 [repeated 4x across cluster]
(ClientAppActor pid=62573) Epoch 2/20
(ClientAppActor pid=62574) Batch 0/638, Loss: 0.1496 [repeated 6x across cluster]
(ClientAppActor pid=62574) Epoch 2/20 [repeated 2x across cluster]
(ClientAppActor pid=62572) Batch 400/668, Loss: 0.0970 [repeated 5x across cluster]
(ClientAppActor pid=62573) Epoch 3/20
(ClientAppActor pid=62572) Batch 0/668, Loss: 0.0405 [repeated 5x across cluster]
(ClientAppActor pid=62574) Epoch 3/20 [repeated 2x across cluster]
(Cl

INFO :      aggregate_fit: received 3 results and 0 failures
ERROR :     ServerApp thread raised an exception: You called `set_weights(weights)` on layer 'simple_nn_1' with a weight list of length 4, but the layer was expecting 0 weights.
ERROR :     Traceback (most recent call last):
  File "/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/flwr/simulation/run_simulation.py", line 268, in server_th_with_start_checks
    updated_context = _run(
  File "/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/flwr/server/run_serverapp.py", line 62, in run
    server_app(grid=grid, context=context)
  File "/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/flwr/server/server_app.py", line 166, in __call__
    start_grid(
  File "/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/flwr/server/compat/app.py", line 90, in start_grid
    hist = run_fl(
  File "/opt/anaconda3/envs/ths_dev/lib/python3.10/site-packages/flwr/server/server.py", line 492, in run_fl
    hist, elap

(ClientAppActor pid=62572) Batch 600/668, Loss: 0.0004


(ClientAppActor pid=62574) 2025-06-06 23:07:39.688440: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro [repeated 2x across cluster]
(ClientAppActor pid=62574) 2025-06-06 23:07:39.688486: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB [repeated 2x across cluster]
(ClientAppActor pid=62574) 2025-06-06 23:07:39.688492: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB [repeated 2x across cluster]
(ClientAppActor pid=62574) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 2x across cluster]
(ClientAppActor pid=62574) I0000 00:00:1749226059.688505 9870246 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support. [repeated 2x across cluster]
(ClientAppActor pid=62574) I0000 00:00:1749226059.688534 9870246 pluggable_device_factory.cc:271] Created TensorFlow device (/job:loca

RuntimeError: Exception in ServerApp thread